# Update_AO_according_to_fiber_innervation
- input
    - AObase_c_g.json
    - AVbase_c_g_f.nrrd
- output
    - AObase_c_g_f_woVC.json

In [10]:
Target_ROI_ID = 672 
Fiber_from ='AI' # injection structure in ID or acronym
Fiber_to = 'CP' # Caudoputamen, id 672, target structure in ID or acronym

dir_data = 'data'
fn_input_AO = 'AObase_c_g.json'
fn_input_AV = 'AVbase_c_g_f.nrrd'
fn_output_AO = 'AObase_c_g_f_woVC.json'

In [9]:
import os
import nrrd
import numpy as np
import pandas as pd
import json
import copy
from collections import OrderedDict
from jsonpath_rw import jsonpath, parse

# Load data

In [11]:
with open(os.path.join(dir_data, fn_input_AO)) as f:
    df_ori = json.load(f, object_pairs_hook=OrderedDict)

In [12]:
AV_ori, header = nrrd.read(os.path.join(dir_data, fn_input_AV))

In [13]:
ID_high_fiber_innervation = AV_ori.max()

# Segment target ROI with high fiber innervation

In [14]:
def Segment_target_ROI_with_high_fiber_innervation_in_AV(match_id, match_fullpath):
    if match_id == 0: return # Children is not defined for a root node.
    # id_offset = 10**9 # ID of segmented node = original ID + this offset
    global df_fiber
    global Target_ROI_ID
    global ID_high_fiber_innervation
    global Fiber_from
    global Fiber_to
    print(match_fullpath)
    print(match_id)
    if match_id == Target_ROI_ID:
        source_acronym =  eval("df_fiber"+\
                str(match_fullpath).replace('.','')\
                .replace('msg', "['msg']").replace('children', "['children']")\
                .replace('id', "['acronym']")) # acronym for matched ID
        source_name =  eval("df_fiber"+\
                str(match_fullpath).replace('.','')\
                .replace('msg', "['msg']").replace('children', "['children']")\
                .replace('id', "['name']")) # name for matched ID
        source_original_content =  eval("copy.deepcopy(df_fiber"+\
                str(match_fullpath).replace('.','')\
                .replace('msg', "['msg']")\
                .replace('children', "['children']").replace('id', "")+")") # OrderedDict
        source_child_index = int(str(match_fullpath)\
            [str(match_fullpath).rfind("[")+1:str(match_fullpath).rfind("]")])
        source_parent_path = "df_fiber"+str(match_fullpath)\
            [0:str(match_fullpath).rfind("[")-1].replace('.','')\
            .replace('msg', "['msg']")\
            .replace('children', "['children']") # ./children[x]→./children
        source_original_voxelcount = eval(("df_fiber"+\
                str(match_fullpath).replace('.','').replace('msg', "['msg']")\
                .replace('children', "['children']").replace('id','')+"['voxel_count']"))
        exec(source_parent_path + "[" + str(source_child_index) +"]['voxel_count'] = None")
        exec(source_parent_path + "[" + str(source_child_index) +"]['acronym'] = '"\
             + source_acronym + "_fiberL'")
        exec(source_parent_path+\
                ".insert(" + str(source_child_index + 1) + ", source_original_content)") 
        exec(source_parent_path + "[" + str(source_child_index + 1) + "]['acronym'] = '"\
             + source_acronym + "_fiberH'")
        exec(source_parent_path + "[" + str(source_child_index + 1) + "]['name'] = '"\
             + source_name +"_fiber_from_"+str(Fiber_from)+'_to_'+str(Fiber_to)+"'")
        exec(source_parent_path + "[" + str(source_child_index + 1) + "]['children'] = []")
        exec(source_parent_path + "[" + str(source_child_index + 1) + "]['id'] = "\
             + str(ID_high_fiber_innervation))
        exec(source_parent_path + "[" + str(source_child_index + 1) +\
             "]['voxel_count'] = " + str(source_original_voxelcount))
        return [match_id, source_acronym, source_name, source_original_voxelcount, ID_high_fiber_innervation] # Leafed ID and its acronym

In [15]:
jsonpath_expr = parse('$..id')
df_fiber = copy.deepcopy(df_ori)
temp_list = []
temp_list = [Segment_target_ROI_with_high_fiber_innervation_in_AV(match.value, match.full_path)\
            for match in reversed(jsonpath_expr.find(df_fiber))]
ID_fiber_high = np.array(temp_list)

msg.[1].id
1000000028
msg.[0].children.[3].id
73
msg.[0].children.[2].id
1000000012
msg.[0].children.[1].id
1009
msg.[0].children.[0].children.[3].id
1000000013
msg.[0].children.[0].children.[2].children.[1].children.[3].id
589508455
msg.[0].children.[0].children.[2].children.[1].children.[2].id
846
msg.[0].children.[0].children.[2].children.[1].children.[1].id
91
msg.[0].children.[0].children.[2].children.[1].children.[0].id
989
msg.[0].children.[0].children.[2].children.[1].id
519
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[5].id
1049
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[4].id
1041
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[3].id
1033
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[2].id
1025
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[1].children.[1].id
1064
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[1].children.[0].id
1056


# Save 

In [16]:
with open(os.path.join(dir_data, fn_output_AO), mode='w') as fw:
    json.dump(df_fiber, fw, indent=4)